In [66]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [67]:
def mask_data_spatial(ds, lon, lat):
    
    spatial_mask = (ds.longitude == lon) * (ds.latitude == lat)

    return ds.where(spatial_mask, drop=True)

In [68]:
lon_Barter_Island = 360 + -143.75
lat_Barter_Island = 70.0

lon_offshore = 360 + -143.75
lat_offshore = 70.25

In [69]:
cwd = os.getcwd()

dir_path = os.path.join(cwd, 'raw_datasets/era5')

save_path = os.path.join(cwd, "")


dir_list = [dir for dir in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, dir))]
offshore_vars = [
    "sea_ice_cover", 
    "sea_surface_temperature", 
    "10m_u_component_of_wind.csv",
    "10m_u_component_of_wind.csv"
]

minyear = 2020
maxyear = 2023

years_with_data = np.arange(minyear, maxyear+1, 1)

locations = []

for dir in dir_list:
    
    time = []
    data = []
       
    for year in years_with_data:
        
        if dir in offshore_vars:
            lon, lat = lon_offshore, lat_offshore
        
        else:
            lon, lat = lon_Barter_Island, lat_Barter_Island
        
        fp = os.path.join(dir_path, dir, dir + "_" + str(year) + ".nc")
        
        ds = xr.open_dataset(fp)
        
        masked_ds = mask_data_spatial(ds, lon, lat).squeeze(drop=True)
        
        colname = list(ds.keys())[0]
                
        ds.close()

        time_year = list(masked_ds.time.values)
       
        time = time + time_year
        
        data += list(masked_ds[colname].values)
        
        masked_ds.close()
            
    df = pd.DataFrame({"time":time})
    df_generated = True
    df[dir] = data
    
    
    if dir in offshore_vars:
        locations.append((dir, lon_offshore, lat_offshore, 'offshore'))
    else:
        locations.append((dir, lon_Barter_Island, lat_Barter_Island, 'onshore'))
        
    df.to_csv(os.path.join("ts_datasets/", dir + ".csv"))
    
    print(f"succesful for variable {dir}")

succesful for variable 10m_u_component_of_wind
succesful for variable 10m_v_component_of_wind
succesful for variable 2m_temperature
succesful for variable mean_surface_latent_heat_flux
succesful for variable mean_surface_net_long_wave_radiation_flux
succesful for variable mean_surface_net_short_wave_radiation_flux
succesful for variable sea_ice_cover
succesful for variable sea_surface_temperature
succesful for variable snow_depth
succesful for variable soil_temperature_level_1
succesful for variable soil_temperature_level_2
succesful for variable soil_temperature_level_3
succesful for variable soil_temperature_level_4


In [70]:
df_loc = pd.DataFrame(data=locations, columns=['var', 'lon', 'lat', 'note'])
